In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler
from utils_robot import *

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

### record position and move to target position

In [2]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        rclpy.spin(self.node)

In [3]:
rclpy.init()
# robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
# jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('posture_control')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)

t1 = Thread(target=ros_interface.spin_thread)
t1.start()

In [4]:
q0=np.array(ros_interface.latest_joint_states.position)
print('current_q:', q0)

current_q: [-1.27526172  0.34152148  0.43530078 -2.30253809  1.22560156 -0.81931934
 -2.02041211  2.81698438  0.        ]


In [5]:
q1=np.array(ros_interface.latest_joint_states.position)
print('current_q:', q1)

current_q: [0.04069973]


In [6]:
target_jp=np.array(q0)

In [7]:
goto_jointpos(ros_interface, target_jp, cmd_pub)

KeyboardInterrupt: 

In [15]:
target_jp.shape

(10,)

In [ ]:
cpos=np.array(ros_interface.latest_joint_states.position)

In [8]:
tpos=np.array(q0)
cpos=np.array(ros_interface.latest_joint_states.position)
dpos=np.linalg.norm(tpos-cpos)
tpos, cpos, dpos

(array([-1.27526172,  0.34152148,  0.43530078, -2.30253809,  1.22560156,
        -0.81931934, -2.02041211,  2.81698438,  0.        ]),
 array([-1.27526172,  0.06153906,  0.30872949, -2.32432031,  1.08205762,
        -0.94028223, -2.02453125,  2.61006055,  0.        ]),
 0.4158800743935447)

In [9]:
def goto_jointpos(target_jointpos, alpha=0.7, tol=0.1,  delay=0.1):
    cpos=np.array(ros_interface.latest_joint_states.position)
    dpos=np.linalg.norm(target_jointpos-cpos)

    while dpos>tol: 
        cpos=np.array(ros_interface.latest_joint_states.position)
        dpos=np.linalg.norm(target_jointpos-cpos)
 
        qd =(target_jointpos-cpos)*alpha

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)

In [10]:
tpos=np.array(q0)
goto_jointpos(tpos)

KeyboardInterrupt: 

In [13]:
target_jointpos=np.array(q0)
tol=0.1
alpha=0.7
delay=0.1

In [18]:
cpos=np.array(ros_interface.latest_joint_states.position)
dpos=np.linalg.norm(target_jointpos-cpos)

while dpos>tol: 
    cpos=np.array(ros_interface.latest_joint_states.position)
    dpos=np.linalg.norm(target_jointpos-cpos)
    print('dpos:', dpos)

    qd =(target_jointpos-cpos)*alpha

    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity = list(qd)
    cmd_pub.publish(cmd_msg)
    time.sleep(delay)

dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 0.4640422028348395
dpos: 4.644408103030912
dpos: 4.644408103030912
dpos: 4.644408103030912
dpos: 4.644408103030912
dpos

KeyboardInterrupt: 

In [15]:
cmd_msg

intera_core_msgs.msg.JointCommand(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1694361752, nanosec=773917860), frame_id=''), mode=2, names=['right_j0', 'head_pan', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6', 'right_gripper_l_finger_joint', 'right_gripper_r_finger_joint'], position=[], velocity=[-0.9211730143749999, 0.2105752278125, 0.276220735625, -1.6402664714062498, 0.8294312825000001, -0.6020133464062499, -1.4427782878125, 1.94339925125, -0.02848981125], acceleration=[], effort=[])